# Table Export Workflow

In [1]:
# Select the kernel observatorio-ipa(Python 3.12.7) .venv/Scripts/python
import json
import ee
import ee.batch
import ee.data
import logging
import importlib
from pathlib import Path
from datetime import date, datetime
from gee_toolbox.gee import assets as toolbox_assets
import sqlite3

from observatorio_ipa.services import connections
from observatorio_ipa.core.workflows import wflows_connections
from observatorio_ipa.core import config
from observatorio_ipa.core.defaults import *
from observatorio_ipa.utils import dates as utils_dates
from observatorio_ipa.services.gee import dates as gee_dates, assets as gee_assets
from observatorio_ipa.services.gee.processes import binary, merge
from observatorio_ipa.core.workflows.images import monthly_export


logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

In [2]:
# general settings
#! JS code shows: users/observatorionieves/Cuencas/Andes # Verified on 2025-06-30

SENSOR = "MCD"
settings = {
    # User and credentials
    "user": "osn-imageautomation-dev@ee-observatorionieves.iam.gserviceaccount.com",
    "service_credentials_file": Path(
        "../secrets/ee-observatorionieves-288939dbc1cf.json"
    ),
    # Image Collections (Used for both Input and Output)
    "monthly_collection_path": Path(
        "projects/ee-observatorionieves/assets/MODIS/Andes_MCDS4S5_Yearly_Monthly"
    ),
    "monthly_image_prefix": "Andes_MCDS4S5_Yearly_Monthly",
    "yearly_collection_path": Path("projects/ee-observatorionieves/assets/MODIS/Andes_MCDS4S5_Yearly"),
    'yearly_image_prefix': "Andes_MCDS4S5_Yearly",

    "aoi_asset_path": Path("projects/ee-observatorionieves/assets/Modules/Andes"),
    "dem_asset_path": Path(
        "projects/ee-observatorionieves/assets/Modules/DEM_SRTM_reproj_MODIS_463_Andes"
    ),


    # ----- Stats Input Assets ------
    "basins_asset_path": Path("users/observatorionieves/DGA/Cuencas_BNA_Oficial"), # BASINS_BNA_ASSET_PATH
    "macrozones_asset_path": Path("users/observatorionieves/DGA/Macrozonas_BNA_Oficial"), # MACROZONAS_BNA_ASSET_PATH
    "basins_cd_property": "COD_CUEN",
    "stats_dem_asset_path": Path("users/observatorionieves/DEM/DEM_SRTM_reproj_Landsat_100_Andes"), # DEM_ASSET_PATH
    "salar_mask_asset_path": Path("projects/ee-observatorionieves/assets/Vectores/Salar_mask"),
    "snow_persistence_asset_path": Path("users/observatorionieves/MODIS/MCD10A1_Andes_T48_Summary/MCD10A1_Andes_T48_Summary_SP"),
    "snow_persistence_trend_asset_path": Path("users/observatorionieves/MODIS/MCD10A1_Andes_T48_Summary/MCD10A1_Andes_T48_Summary_ST"),

    # ----- Stats Common Output Config -----
    "stats_export_target": "storage",
    "stats_storage_bucket": "chompitest-odes",
    "stats_base_export_path": Path("OSN/stats/"),
    "basin_codes": ['023', '024', '030'],
    "exclude_basin_codes": None,
    "max_exports": None,
    "common_tbl_pre_prefix": "MCD",
    # ----- Elevation Output -----
    'elevation_tbl_export_path': Path("elev_ee"),
    'elev_basin_tbl_prefix': f"elev_BNA_",
    'sca_elev_basin_tbl_prefix': f"SCA_elev_BNA_",

    # ----- Month (across-years) Statistics Output -----
    'month_tbl_export_path': Path("month_ee"),
    'sca_m_basin_tbl_prefix': f"SCA_m_BNA_",
    'sca_m_elev_basin_tbl_prefix': f"SCA_m_elev_BNA_",
    'sca_m_trend_basin_tbl_prefix': f"SCA_m_trend_BNA_",

    # ----- Monthly (year/month) Statistics Output -----
    'year_month_tbl_export_path': Path("yearMonth_ee"),
    'sca_y_m_basin_tbl_prefix': f"SCA_y_m_BNA_",
    'sca_ym_basin_tbl_prefix': f"SCA_ym_BNA_",
    'sca_ym_elev_basin_tbl_prefix': f"SCA_ym_elev_BNA_",
    'snowline_ym_basin_tbl_prefix': f"snowline_ym_BNA_",

    # ----- Yearly Statistics Output -----
    'year_tbl_export_path': Path("year_ee"),
    'sca_y_basin_tbl_prefix': f"SCA_y_BNA_",
    'sca_y_elev_basin_tbl_prefix': f"SCA_y_elev_BNA_",
    'sca_y_t_area_basin_tbl_prefix': f"SCA_y_t_area_BNA_",
    'sca_y_t_elev_basin_tbl_prefix': f"SCA_y_t_elev_BNA_",
    'snowline_y_basin_tbl_prefix': f"snowline_y_BNA_",

    # ----- stats manifest -----
    "manifest_source": "file",  # file or storage
    "monthly_manifest_path": Path("../manifests/"),
}

In [3]:
# Connect to Google Earth Engine
from observatorio_ipa.services import connections

runtime_service_account = connections.GoogleServiceAccount(
    settings["service_credentials_file"].as_posix(),
)
connections.connect_to_gee(runtime_service_account)

In [4]:
# Connect to Google Cloud Storage
from google.oauth2 import service_account
from googleapiclient.errors import HttpError

# Make Google Cloud Credentials
# Connect to Google API and build service
google_credentials = service_account.Credentials.from_service_account_info(
    runtime_service_account.credentials
)

In [5]:
from observatorio_ipa.core.workflows.tables import monthly_exports
importlib.reload(monthly_exports)

<module 'observatorio_ipa.core.workflows.tables.monthly_exports' from 'C:\\Users\\erick\\Documents\\Projects\\teleamb\\observatorio_ipa\\src\\observatorio_ipa\\core\\workflows\\tables\\monthly_exports.py'>

In [6]:
# Create Google Cloud Storage client
storage_client = monthly_exports.create_storage_client(
    project=runtime_service_account.project_id,
    credentials=google_credentials
)


In [7]:
monthly_export_list = monthly_exports.monthly_tbl_export_proc(
    settings=settings, 
    storage_conn=storage_client,
    storage_bucket=settings["stats_storage_bucket"],
    force_overwrite=True,
    skip_manifest=False
)

Table Names Prefix:
 - MCD_elev_BNA_
 - MCD_SCA_elev_BNA_
 - MCD_SCA_m_BNA_
 - MCD_SCA_m_elev_BNA_
 - MCD_SCA_m_trend_BNA_
 - MCD_SCA_y_m_BNA_
 - MCD_SCA_ym_BNA_
 - MCD_SCA_ym_elev_BNA_
 - MCD_snowline_ym_BNA_
 - MCD_SCA_y_BNA_
 - MCD_SCA_y_elev_BNA_
 - MCD_SCA_y_t_area_BNA_
 - MCD_SCA_y_t_elev_BNA_
 - MCD_snowline_y_BNA_
Elevation Export Path: OSN\stats\elev_ee
Elevation Archive Path: OSN\stats\archive\elev_ee
Moved and renamed OSN/stats/elev_ee/MCD_SCA_elev_BNA_023.csv to OSN/stats/archive/elev_ee/MCD_SCA_elev_BNA_023_LU20250826.csv
Moved and renamed OSN/stats/elev_ee/MCD_SCA_elev_BNA_024.csv to OSN/stats/archive/elev_ee/MCD_SCA_elev_BNA_024_LU20250826.csv
Moved and renamed OSN/stats/elev_ee/MCD_SCA_elev_BNA_030.csv to OSN/stats/archive/elev_ee/MCD_SCA_elev_BNA_030_LU20250826.csv
Month Export Path: OSN\stats\month_ee
Month Archive Path: OSN\stats\archive\month_ee
Monthly Export Path: OSN\stats\yearMonth_ee
Monthly Archive Path: OSN\stats\archive\yearMonth_ee
Moved and renamed OSN/sta

In [16]:
monthly_export_list.query_status()

{'COMPLETED': 24}

In [17]:
print(monthly_export_list.pretty_summary())

| Status    | Count |
| COMPLETED |   24  |
